In [2]:
import json
import networkx as nx
import numpy as np
import random
from collections import Counter

In [3]:
def calculateProb(item) :
    # preference
    if item == 0 or item == 1 :
        return 1
    # perception of preference
    elif item == 2 or item == 3 :
        return 0.7
    # avoidance
    elif item == 4 or item == 5 :
        return 0
    # perception of avoidance
    else :
        return 0.3

In [4]:
def findProbBetween(prefGraph, node1, node2) :
        # get weights of edges from node1 to node2
        edges_node1_to_node2 = [calculateProb(data["questionItem"]) for u, v, data in prefGraph.edges(node1, data=True) if v == node2]
        
        # get weights of edges from node2 to node1
        edges_node2_to_node1 = [calculateProb(data["questionItem"]) for u, v, data in prefGraph.edges(node2, data=True) if v == node1]

        if len(edges_node1_to_node2) == 0 and len(edges_node2_to_node1) == 0 :
            # no opinion/thought between node1 & node2
            return None
        elif len(edges_node1_to_node2) == 0 :
            # only node2 has opinion/thought for node1
            return np.average(edges_node2_to_node1)
        elif len(edges_node2_to_node1) == 0 :
            # only node1 has opinion/thought for node2
            return np.average(edges_node1_to_node2)
        else :
            # both node1 and node2 share opinions/thoughts for each other
            return np.average([np.average(edges_node1_to_node2), np.average(edges_node2_to_node1)])  

In [5]:
def findPossiblePeers(prefGraph, threshold, numNodes) :    
    # dictionary {edge : probability}
    edgesProbabilities = {}
    # examine edges between nodes in Preference Graph
    # calculate probability. How strong is the connection between people?
    for node in range(numNodes) :
        for other in range(node + 1, numNodes) :
            prob = findProbBetween(prefGraph, node, other)
            if prob is not None :
                edgesProbabilities[(node, other)] = prob

    # find possible peers for a student
    # step 1: find connecting links with prob < threshold
    possiblePeers = [[] for _ in range(numNodes)]       
    for edge, prob in edgesProbabilities.items() :
        if prob < threshold :
            possiblePeers[edge[0]].append(edge[1])
            possiblePeers[edge[1]].append(edge[0])

    # step 2: keep all the other nodes as possible peers to interact with
    possiblePeers = list(map(lambda item : list(set(range(numNodes)) - set(item[1] + [item[0]])), enumerate(possiblePeers)))
    
    for node, peers in enumerate(possiblePeers) :
        #  dictionary for node {peer: prob between node and peer}
        d = dict()
        for peer in peers :
            try :
                d[peer] = edgesProbabilities[(min(node, peer), max(node, peer))]
            except :
                d[peer] = 1 
        possiblePeers[node] = d
    return possiblePeers

In [6]:
def createGraphSnapshot(numNodes, maxInteractions, possiblePeers) :
    G = nx.Graph()
    G.add_nodes_from(range(numNodes))
    nodes = list(range(numNodes))
    random.shuffle(nodes)
    for node in nodes :
        # achievablePeers: >= threshold, enough capacity, not already connected
        achievablePeers = [u for u in possiblePeers[node] if maxInteractions > G.degree(u) and u not in G.neighbors(node)]
        degrees = [G.degree(u) for u in achievablePeers]
        
        # sort achievablePeers based on their degree
        achievablePeers = [node for _, node in sorted(zip(degrees, achievablePeers))]

        # interactions are maximum 3
        # restricted further by degree of current node or number of available peers
        interactions = min(3, maxInteractions-G.degree(node), len(achievablePeers))
       
        if interactions == 0 :
            continue
        else :
            # get the favored, minimum-connected nodes
            peers = achievablePeers[:interactions]

            # for current node
            # find A edges, B edges, C edges
            counter = dict.fromkeys(["A", "B", "C"], 0)
            counter.update(Counter([e[2]["weight"] for e in G.edges(node, data=True)]))
            
            # sort A, B, C based on this counting
            # get the types of activity in shortage for current node
            weights = [key for key, value in sorted(counter.items(), key=lambda item: item[1])][:interactions]

            # create new edges with weights A, B, C
            for peer, weight in zip(peers, weights) :
                G.add_edge(node, peer, weight=weight)


    for node in range(numNodes) :
        G.nodes[node]["A"] = None
        G.nodes[node]["B"] = None
        G.nodes[node]["C"] = None

    activitiesIndex = {"A": [1, 2], "B": [3, 4], "C": [5, 6]}
    edgesToRemove = []

    for u, v, d in G.edges(data="weight") :
        # current edge
        # get weights of adjacent nodes for activity type d
        node1 = G.nodes[u][d]
        node2 = G.nodes[v][d]
        if node1 is None and node2 is None :
            # no activity of type d in nodes
            # assign random activity to nodes, edge
            activity = np.random.choice(activitiesIndex[d])
            G.nodes[u][d] = activity
            G.nodes[v][d] = activity
            G[u][v]["weight"] = activity
        elif node1 is not None and node2 is not None:
            # activity of type d exists in nodes
            # only if it's the same, assign the same to edge
            # else remove edge
            if node1 == node2 :
                G[u][v]["weight"] = node1
            else :
                edgesToRemove.append((u,v))
        else :
            # activity of type d only for one node
            # assign the same to other node, edge
            activity = node1 or node2    
            G.nodes[u][d] = activity
            G.nodes[v][d] = activity
            G[u][v]["weight"] = activity 

    G.remove_edges_from(edgesToRemove)
   
    return G

In [7]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    """creates an Interaction Graph for formation of groups,
       assignation of activities &
       creation of studentsɐ interactions

    Parameters
    ----------
    preferenceGraph : MultiDiGraph object
        graph of opinions & thoughts in a student community
    threshold : real
        minimum probability value for students to engage together
    maxInteractions : int
        maximum number of interactions for each node in a snapshot
    maxTimestamp : int
        maximum number of snapshots to be created
    Raises
    ------
    ValueError
        If maxInteractions is less than 3 and greater than n-1.
    Returns
    -------
    dictionary => "graph" : list of Graph objects
        (maxTimestamp + 1) undirected graph objects, independent between them
    """

    n = preferenceGraph.number_of_nodes()

    if maxInteractions <= 2 or maxInteractions >= n:
        raise ValueError(f"maxInteractions must be 3 ≤ maxInteractions ≤ {n-1}")

    interactionGraph = []
   
    # list of possible peers to inteact for each person
    possiblePeers = findPossiblePeers(prefGraph=preferenceGraph, threshold=threshold, numNodes=n)
    
    # interactions at timestamp t = 0
    # 1st step: interactions based on a scale-free network model with m=3
    interactionGraph.append(nx.barabasi_albert_graph(n=n, m=3))

    # interactions at timestamps t >= 1
    for time in range(1, maxTimestamp + 1) :
        # create new snapshot
        snapshot = createGraphSnapshot(numNodes=n, maxInteractions=maxInteractions, possiblePeers=possiblePeers)
        interactionGraph.append(snapshot)

    return {"graph" : interactionGraph}

In [8]:
# function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open(filePath) as file :
        data = json.load(file)

    G = nx.MultiDiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], opinion=edge["opinion"], questionItem=edge["opinion item"])
    
    return G

In [ ]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.4, # threshold for probability
    "maxInteractions" : 4, # maximum possible number of interactions for a node
    "maxTimestamp" : 15 # number of snapshots
}

# create interaction graph
G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)["graph"] # G is a list of networkx Graphs